In [7]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime

# display option (float 자리수)
pd.options.display.float_format = lambda x: f'{x:.4f}'


### 1. 데이터 가져오기
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

# PNU별 매출액
sql = '''
select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00'
  from m1.kt_bldg_sales
 where base_ym = '202209'
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kt_bldg_sales = pd.DataFrame(rows)
df_kt_bldg_sales.columns = ['pnu','sales','industry_cd','emd_cd']

# 법정동별 소득 
sql = '''
select cur_lgl_dong_cd||'00' as emd_nm, avg(avg_mon_income) as income_avg
  from m1.kcb_stat_cm
 where bs_yr_mon = '202209'
 group by cur_lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_stat_cm = pd.DataFrame(rows)
df_kcb_stat_cm.columns = ['emd_cd','income_avg']

MemoryError: 

In [ ]:
# PNU별 매출액
sql = '''
select substring (bd_mgt_sn, 0, 20) as pnu, pred_slng_amt as sales, ftc_cate2_cd, emd_cd||'00'
  from m1.kt_bldg_sales
 where base_ym = '202209'
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kt_bldg_sales = pd.DataFrame(rows)
df_kt_bldg_sales.columns = ['pnu','sales','industry_cd','emd_cd']

In [ ]:
# 법정동별 소득 
sql = '''
select cur_lgl_dong_cd||'00' as emd_nm, avg(avg_mon_income) as income_avg
  from m1.kcb_stat_cm
 where bs_yr_mon = '202209'
 group by cur_lgl_dong_cd
'''
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_kcb_stat_cm = pd.DataFrame(rows)
df_kcb_stat_cm.columns = ['emd_cd','income_avg']

In [ ]:
df_kt_bldg_sales

In [ ]:
df_kcb_stat_cm

In [ ]:
# 데이터 프레임 병합
df = pd.merge(df_kt_bldg_sales, df_kcb_stat_cm, on='emd_cd', how='left')

# 소득이 0명, null인 지역 제외
df = df[~df['income_avg'].isnull()]

# 매출액 / 소득
df['income_per_sales'] = (df['sales_avg'] / df['income_avg']) *1000 # 원(WON) 단위로 변경
df = df[['pnu','emd_cd','income_per_sales','industry_cd']]

# 작업일(업데이트일) 및 작업자 추가
df['create_at'] = datetime.today().strftime("%Y-%m-%d")
df['update_at'] = datetime.today().strftime("%Y-%m-%d")
df['work_user'] = 'du.Park'

In [ ]:
df

In [ ]:
df['income_per_salse'].median()

In [27]:
### 2. DB INSERT
import datetime

table = 'm2.datadam_income_per_sales'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

2022-10-30 20:16:04.949272  :  0
4938


In [26]:
df

,emd_cd,income_per_sales,base_ym
0,1111010400,16912.5494,202210
1,1111010700,39912.6109,202210
2,1111010800,20036.1000,202210
3,1111010900,13211.3150,202210
4,1111011000,31967.1544,202210
...,...,...,...
4934,4617012900,7119.1176,202210
4935,4219011000,5318.6528,202210
4936,4518011100,30750.3650,202210
4937,3611011700,13238.9349,202210


In [ ]:
df['income_per_sales'].median()

In [25]:
df.describe()

,income_per_sales
count,4938.0000
mean,37122.1020
std,32980.0333
min,0.0000
25%,21607.0372
50%,31498.3104
75%,42490.2274
max,660444.6672
